In [ ]:
### Read PDF

# from dotenv import load_dotenv
# Charger les variables d'environnement
# load_dotenv()
# Faire lettre motivation automatique

import re

In [5]:
from PyPDF2 import PdfReader

def read_pdf(file_path: str) -> str:
    """Lit et extrait le texte d'un CV au format PDF d'une seule page.

    Args:
        file_path (str): Chemin d'accès vers le fichier PDF à lire

    Returns:
        str: Texte brut extrait du PDF

    Raises:
        FileNotFoundError: Si le fichier PDF n'existe pas
        IndexError: Si le PDF est vide
        Exception: Pour toute autre erreur lors de la lecture du PDF
    """
    try:
        reader = PdfReader(file_path)
        if len(reader.pages) == 0:
            raise IndexError("Le PDF est vide")
        
        page = reader.pages[0]  # Lecture de la première page uniquement
        text_brut = page.extract_text()
        
        if not text_brut:
            raise Exception("Aucun texte n'a pu être extrait du PDF")
            
        return text_brut

    except FileNotFoundError:
        raise FileNotFoundError(f"Le fichier {file_path} n'existe pas")
    except IndexError as e:
        raise IndexError(str(e))
    except Exception as e:
        raise Exception(f"Erreur lors de la lecture du PDF: {str(e)}")



In [6]:
reader = PdfReader("CV_V4_EN.pdf")
page = reader.pages[0] # 1 seule page
text_brut = page.extract_text()
print(text_brut)
print(type(text_brut))
print(len(text_brut))

Quentin Lim  
Last year Student in 
Data Science
quentinlim 384@ gm ail.com
+33782885515
linkedin.com /in/quentin-lim-
978746250
github.com /QL2111
Profile
Currently seeking an internship in 
Data Science starting in March 
2025 for a duration of 4 to 6 
months.
Skills
M L, NLP, TensorFlow; Scikit-learn, 
M LFlow, Transform ers, LLM , Tim e 
Series.
SQL, NoSQL, M ongoDB
Qlik, Tableau, PowerBI
OpenCV, YOLO
Azure, AW S, Fabric
Git/Agile m ethodology
Python/R/Java/PySpark/JavaScript/
PHP
Gen AI
Interests
Board Games: Casual boarding
gam es and Chess club
Gaming: Played in competitives
tournam ents, peaked GrandMaster
in League of Legends
Sailing: M em ber of the sailing club
at La Rochelle Université
Basket ball: Sports association
during M iddle School, teamwork
and fast decision making.
Volunteering
ESN Cosmolyon
2023 – present
Vice-President, holdings Culturals 
Event for internationals and local 
student(Board gam es/Day 
trips/Spanish dinner)
Alter-ego Lyon 2
2023 – present
Buddy Sys

In [7]:
# print(read_pdf("CV_V4_EN.pdf"))

### Model Mistral Mini -> Reformulation Texte

In [8]:
# TODO
# Formulation du prompt (CV texte brut + )
# Ajouter les suivis de prix + impact ecologique ?


In [9]:
import os
import litellm
import json
from dotenv import load_dotenv, find_dotenv


load_dotenv()



True

In [10]:
reformulation_prompt = f"""
A partir d'un texte brut venant d'un CV, extrait les informations suivantes au format JSON :
{{
    "Diplome": [
      {{
          "niveau_etudes": "int",
          "domaine_etudes": ["str"]
      }}
    ],

    "Competences": ["str"],
    "Experiences": [
        {{
            "domaine_activite": ["str"],
            "poste_occupe": "str",
            "duree": "int"
        }}
    ],
    "Profil": {{
        "titre": "str",
        "disponibilite": "str"
    }}
}}

Texte du CV :
"{text_brut}"

Ne retourne que le JSON
"""
reformulation_prompt

'\nA partir d\'un texte brut venant d\'un CV, extrait les informations suivantes au format JSON :\n{\n    "Diplome": [\n      {\n          "niveau_etudes": "int",\n          "domaine_etudes": ["str"]\n      }\n    ],\n\n    "Competences": ["str"],\n    "Experiences": [\n        {\n            "domaine_activite": ["str"],\n            "poste_occupe": "str",\n            "duree": "int"\n        }\n    ],\n    "Profil": {\n        "titre": "str",\n        "disponibilite": "str"\n    }\n}\n\nTexte du CV :\n"Quentin Lim  \nLast year Student in \nData Science\nquentinlim 384@ gm ail.com\n+33782885515\nlinkedin.com /in/quentin-lim-\n978746250\ngithub.com /QL2111\nProfile\nCurrently seeking an internship in \nData Science starting in March \n2025 for a duration of 4 to 6 \nmonths.\nSkills\nM L, NLP, TensorFlow; Scikit-learn, \nM LFlow, Transform ers, LLM , Tim e \nSeries.\nSQL, NoSQL, M ongoDB\nQlik, Tableau, PowerBI\nOpenCV, YOLO\nAzure, AW S, Fabric\nGit/Agile m ethodology\nPython/R/Java/PyS

In [11]:
# Vérification de la clé API
#  api_key=os.getenv("MISTRAL_API_KEY")
# api_key

In [12]:
# litellm._turn_on_debug()
# Température, plus c'est proche de 1, plus c'est créatif
# On cherche à savoir combien de token on devrait mettre

CV_reformuler = litellm.completion(
            model="mistral/ministral-3b-latest",  # Choix d'un modèle petit
            messages=[{"role": "user", "content": reformulation_prompt}],
            max_tokens=1500,
            temperature=0.3,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

resultat = CV_reformuler["choices"][0]["message"][
            "content"
        ].strip()

In [13]:
# Le CV fait 4231 en len
# len de 1863 pour 500 max tokens
# len de 4429 pour 1500 max tokens -> C'est OK, on essaye avec 1000 max tokens
print(resultat)
print(len(resultat))



```json
{
    "Diplome": [
        {
            "niveau_etudes": "Master",
            "domaine_etudes": ["Data Science"]
        }
    ],
    "Competences": [
        "M L, NLP, TensorFlow; Scikit-learn, M LFlow, Transform ers, LLM , Tim e Series.",
        "SQL, NoSQL, M ongoDB",
        "Qlik, Tableau, PowerBI",
        "OpenCV, YOLO",
        "Azure, AW S, Fabric",
        "Git/Agile m ethodology",
        "Python/R/Java/PySpark/JavaScript/",
        "PHP",
        "Gen AI"
    ],
    "Experiences": [
        {
            "domaine_activite": ["Data Science"],
            "poste_occupe": "Student Job at the IT department of the university",
            "duree": 1
        },
        {
            "domaine_activite": ["Data Science"],
            "poste_occupe": "Internship Data Analyst",
            "duree": 1
        }
    ],
    "Profil": {
        "titre": "Currently seeking an internship in Data Science starting in March 2025 for a duration of 4 to 6 months.",
        "disponib

In [ ]:
def analyze_cv(text_brut: str, temperature: float = 0.01, max_tokens: int = 1500) -> str:
    """
    Analyse un CV en format texte et retourne les informations structurées en JSON.

    Args:
        text_brut (str): Texte brut du CV à analyser
        temperature (float, optional): Paramètre de créativité du modèle. Defaults to 0.2.
        max_tokens (int, optional): Nombre maximum de tokens pour la réponse. Defaults to 1500.

    Returns:
        str: Informations en string(réponse du LLM) structurées du CV au format JSON

    Raises:
        Exception: En cas d'erreur d'API ou de parsing JSON
    """
    try:
        reformulation_prompt = f"""
        À partir du texte brut d'un CV, extrais les informations suivantes au format JSON. 
        Si une information n'est pas explicitement mentionnée dans le texte, retourne `null` ou une liste vide.
        Ne devine pas les informations manquantes et ne retourne que ce qui est clairement présent dans le texte.

        Format JSON attendu :
        {{
            "Formation": [
                {{
                    "niveau_etudes": "str",  // Ex: "Licence", "Master", "Doctorat"
                    "domaine_etudes": ["str"]  // Ex: ["Informatique", "Mathématiques"]
                }}
            ],
            "Competences": ["str"],  // Ex: ["Python", "Gestion de projet"]
            "Experiences": [
                {{
                    "domaine_activite": ["str"],  // Ex: ["Tech", "Finance"]
                    "poste_occupe": "str",  // Ex: "Développeur Python"
                    "duree": "str"  // Ex: "2 ans"
                }}
            ],
            "Profil": {{
                "titre": "str",  // Ex: "Développeur Full-Stack"
                "disponibilite": "str"  // Ex: "dd-mm-yyyy"
            }}
        }}

        Texte du CV :
        "{text_brut}"

        Ne retourne que le JSON, sans commentaires supplémentaires.
        """

        CV_reformuler = litellm.completion(
            model="mistral/mistral-medium",
            messages=[{"role": "user", "content": reformulation_prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

        # Extraire et parser le JSON de la réponse
        resultat = CV_reformuler["choices"][0]["message"]["content"].strip()
        return resultat
    
    except Exception as e:
        raise Exception(f"Erreur lors de l'analyse du CV: {str(e)}")

In [15]:
CV_reformuler = analyze_cv(text_brut)
print(CV_reformuler)

{
  "Formation": [
    {
      "niveau_etudes": "Master",
      "domaine_etudes": ["Data Science"]
    }
  ],
  "Competences": [
    "ML",
    "NLP",
    "TensorFlow",
    "Scikit-learn",
    "MLFlow",
    "Transformers",
    "LLM",
    "Time Series",
    "SQL",
    "NoSQL",
    "MongoDB",
    "Qlik",
    "Tableau",
    "PowerBI",
    "OpenCV",
    "YOLO",
    "Azure",
    "AWS",
    "Fabric",
    "Git/Agile methodology",
    "Python",
    "R",
    "Java",
    "PySpark",
    "JavaScript",
    "PHP",
    "Gen AI"
  ],
  "Experiences": [
    {
      "domaine_activite": [],
      "poste_occupe": "Vice-President",
      "duree": "2023 – present"
    },
    {
      "domaine_activite": [],
      "poste_occupe": "Buddy System",
      "duree": "2023 – present"
    },
    {
      "domaine_activite": ["Data Science"],
      "poste_occupe": "Student Job at the IT department",
      "duree": "present"
    },
    {
      "domaine_activite": ["Data Science"],
      "poste_occupe": "Internship Data A

#### Matching CV et offre :

- Obtention du string job offer par le scrapping
- Transformation via un LLM de l'offre à un format similaire au CV_reformuler
- Embedding + calcul de la similarité cosinus globale
- Transformation en JSON du CV_reformuler et job_offer_reforumuler pour pouvoir effectuer une similarité par partie(clef communes car passé tout les deux par un LLM)
- Calcul des similarité par parties

In [16]:



# Example job offer
job_offer = """
  Titre : Data scientist (H/F) SALON TAF Toulouse 2025
  Entreprise : AOSIS CONSULTING
  Contrat : CDI
  Localisation : 31 - TOULOUSE
  Description :
  Concrètement qu'est-ce qu'on fait ?
  Nous proposons des Services, des Logiciels et de la Formation dans le domaine de la Business Intelligence, de la Data Science, de la Gouvernance et de la Visualisation de la donnée. L'Infrastructure et le Développement font également partie du panel de nos compétences.
  Notre leitmotiv ? Citoyenneté, écologie et sport avec des experts de la donnée.
  Notre objectif ? Porter encore plus haut et plus fort nos valeurs de sport, d'écologie et de citoyenneté et accompagner nos consultants dans leur montée en compétence.

  Le poste
  Dans le cadre d'une ouverture de poste, nous sommes à la recherche d'un.e Data Scientist pour l'un de nos clients.
  Un.e data scientist senior (5 ans d'expérience) très autonome dans ses analyses et explorations, expérimenté dans l'industrialisation des modèles dans des environnements cloud.

  Vous interviendrez sur toutes les étapes du projet, vous devrez :

  Identifier leurs difficultés et proposer des solutions
  Evaluer la qualité des données, les nettoyer, les agréger, et mener des études adhoc
  Concevoir, implémenter et comparer différents algorithmes et méthodes statistiques
  Concevoir et mener des protocoles de test en conditions réelles (magasins, entrepôts, .)
  Développer des outils de visualisation des données
  Mettre en production, maintenir et itérer sur les solutions

  Environnement technique: Python, SQL, Terraform, Bitbucket, Jenkins, Airflow, Docker, Kubernetes, GCP (BigQuery, Spark DataProc)

  Profil recherché
  Vous êtes issu.e d'un Master en informatique ou mathématiques appliquées.
  Vous avez à minima 5 années d'expériences en Data Science et maitrisez les algorithmes d'apprentissage statistique.
  Vous maîtrisez Python et SQL
  Vous avez une expérience dans le développement logiciel agile avec multiples contributeurs
  Vous avez une expérience du cloud, idéalement GCP

  Pourquoi nous rejoindre ?
  Parce que nous sommes une société à taille humaine
  Parce que nous nous engageons (réellement) dans des enjeux importants : l'écologie, la citoyenneté et le sport
  Mais aussi parce que nous aimons réaliser tout un tas d'activités : des jeux, des afterworks, des restaurants, ... et plein d'autres choses encore :)
"""


#### Reformulation de l'offre d'emploi :

In [ ]:
#### Transformation de l'offre d'emploi au même format que le CV
def analyze_cv_offre_emploi(offre: str, temperature: float = 0.01, max_tokens: int = 1500) -> str:
    """
    Analyse une offre d'emploi en format texte et retourne les informations structurées en JSON.

    Args:
        text_brut (str): Texte brut del'offre à analyser
        temperature (float, optional): Paramètre de créativité du modèle. Defaults to 0.2.
        max_tokens (int, optional): Nombre maximum de tokens pour la réponse. Defaults to 1500.

    Returns:
        str: Informations en string(réponse du LLM) structurées du CV au format JSON

    Raises:
        Exception: En cas d'erreur d'API ou de parsing JSON
    """
    try:
        reformulation_prompt = f"""
        À partir de cette offre d'emploi, extrais les informations suivantes au format JSON. 
        Si une information n'est pas explicitement mentionnée dans le texte, retourne `null` ou une liste vide.
        Ne devine pas les informations manquantes et ne retourne que ce qui est clairement présent dans le texte.

        Format JSON attendu :
        {{
            "Formation": [
                {{
                    "niveau_etudes": "str",  // Ex: "Licence", "Master", "Doctorat"
                    "domaine_etudes": ["str"]  // Ex: ["Informatique", "Mathématiques"]
                }}
            ],
            "Competences": ["str"],  // Ex: ["Python", "Gestion de projet"]
            "Experiences": [
                {{
                    "domaine_activite": ["str"],  // Ex: ["Tech", "Finance"]
                    "poste_occupe": "str",  // Ex: "Développeur Python"
                    "duree": "str"  // Ex: "2 ans"
                }}
            ],
            "Profil": {{
                "titre": "str",  // Ex: "Développeur Full-Stack"
                "disponibilite": "str"  // Ex: "dd-mm-yyyy"
            }}
        }}

        Texte de l'offre d'emploi :
        "{offre}"

        Ne retourne que le JSON, sans commentaires supplémentaires.
        """

        offre_reformuler = litellm.completion(
            model="mistral/mistral-medium",
            messages=[{"role": "user", "content": reformulation_prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

        # Extraire et parser le JSON de la réponse
        resultat = offre_reformuler["choices"][0]["message"]["content"].strip()
        return resultat
    
    except Exception as e:
        raise Exception(f"Erreur lors de l'analyse du CV: {str(e)}")

In [18]:
job_offer_reforumer = analyze_cv_offre_emploi(job_offer)
print(job_offer_reforumer)

{
  "Formation": [
    {
      "niveau_etudes": "Master",
      "domaine_etudes": ["Informatique", "Mathématiques appliquées"]
    }
  ],
  "Competences": ["Python", "SQL", "Terraform", "Bitbucket", "Jenkins", "Airflow", "Docker", "Kubernetes", "GCP (BigQuery, Spark DataProc)", "Apprentissage statistique", "Développement logiciel agile"],
  "Experiences": [
    {
      "domaine_activite": ["Data Science"],
      "poste_occupe": "Data Scientist",
      "duree": "5 ans"
    }
  ],
  "Profil": {
    "titre": "Data Scientist",
    "disponibilite": null
  }
}


#### Similarité cosinus globale


In [30]:
# print(HF_TOKEN)

In [31]:
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Token d'API Hugging Face
HF_TOKEN = os.getenv("HF_TOKEN")
API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"


def get_embedding(text: str) -> np.ndarray:
    """
    Envoie une requête à l'API Hugging Face pour obtenir l'embedding d'un texte.

    Args:
        text (str): Texte à encoder.

    Returns:
        np.ndarray: Embedding du texte.
    """
    headers = {"Authorization": f"Bearer {os.getenv("HF_TOKEN")}"}
    response = requests.post(
        "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        headers=headers,
        json={"inputs": text, "options": {"wait_for_model": True}},
    )
    if response.status_code != 200:
        raise Exception(f"Erreur API : {response.status_code}, {response.text}")
    return np.array(response.json())

In [32]:
import numpy as np

def calculate_similarity(text1: str, text2: str) -> float:
    """
    Calcule la similarité cosinus entre deux textes en utilisant l'API Hugging Face.
    
    Args:
        text1 (str): Premier texte.
        text2 (str): Deuxième texte.
    
    Returns:
        float: Score de similarité cosinus entre les deux textes.
    """
    # Obtenir les embeddings des textes
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    # Calculer la similarité cosinus
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

# Calculer la similarité entre le CV et l'offre d'emploi
similarity_score = calculate_similarity(CV_reformuler, job_offer_reforumer)
print(f"Similarité globale entre le CV et l'offre d'emploi: {similarity_score:.4f}")


Similarité globale entre le CV et l'offre d'emploi: 0.9035


#### Conversion en JSON-like

In [21]:

# Calcul de la similarité entre chaque partie du CV et l'offre d'emploi
# Diviser le CV reformulé en json en plusieurs parties
cv_json = json.loads(CV_reformuler)
# print(cv_json)
# convert job_offer to json
job_offer_json = json.loads(job_offer_reforumer)

In [22]:
print(job_offer_json)

{'Formation': [{'niveau_etudes': 'Master', 'domaine_etudes': ['Informatique', 'Mathématiques appliquées']}], 'Competences': ['Python', 'SQL', 'Terraform', 'Bitbucket', 'Jenkins', 'Airflow', 'Docker', 'Kubernetes', 'GCP (BigQuery, Spark DataProc)', 'Apprentissage statistique', 'Développement logiciel agile'], 'Experiences': [{'domaine_activite': ['Data Science'], 'poste_occupe': 'Data Scientist', 'duree': '5 ans'}], 'Profil': {'titre': 'Data Scientist', 'disponibilite': None}}


In [23]:
print(type(cv_json)) # Les données sont en string JSON like
print(type(job_offer_json)) # Les données sont en string JSON like

<class 'dict'>
<class 'dict'>


#### Calcul des similarités cosinus par parties :

- Formation
- Competences
- Experiences
- Profil

In [24]:
# Similarité entre les deux json like

# Similarité Formation
# print(cv_json.get("Formation"))
# print(job_offer_json.get("Formation"))


In [25]:
# Similarité entre Formation attendu et Formation du CV
formation_candidat = cv_json.get("Formation")
formation_attendu = job_offer_json.get("Formation")

similarite_formation = calculate_similarity(str(formation_candidat), str(formation_attendu))
print(f"Similarité entre Formation attendu et Formation du CV: {similarite_formation:.4f}")


# Similarité entre Compétences attendu et Compétences du CV
competences_candidat = cv_json.get("Competences")
# print(competences_candidat)
competences_attendu = job_offer_json.get("Competences")
# print(competences_attendu)

similarite_competences = calculate_similarity(str(competences_candidat), str(competences_attendu))
print(f"Similarité entre Compétences attendu et Compétences du CV: {similarite_competences:.4f}")


# Similarité entre Experiences attendu et Experiences du CV
experiences_candidat = cv_json.get("Experiences")
# print(experiences_candidat)
experiences_attendu = job_offer_json.get("Experiences")
# print(experiences_attendu)

similarite_experiences = calculate_similarity(str(experiences_candidat), str(experiences_attendu))
print(f"Similarité entre Experiences attendu et Experiences du CV: {similarite_experiences:.4f}")


# Similarité entre Profil attendu et Profil du CV
profil_candidat = cv_json.get("Profil")
print(profil_candidat)
profil_attendu = job_offer_json.get("Profil")
print(profil_attendu)

similarite_profil = calculate_similarity(str(profil_candidat), str(profil_attendu))
print(f"Similarité entre Profil attendu et Profil du CV: {similarite_profil:.4f}")

###################### Voir si on ne peut pas faire mieux (check if any au lieu d'une simalarité globale) ######################
### Au lieu d'installer sentence transformers, utiliser une API huggingface avec pipeline et requests


Similarité entre Formation attendu et Formation du CV: 0.7965
Similarité entre Compétences attendu et Compétences du CV: 0.8027
Similarité entre Experiences attendu et Experiences du CV: 0.6901
{'titre': 'Currently seeking an internship in Data Science', 'disponibilite': 'starting in March 2025 for a duration of 4 to 6 months.'}
{'titre': 'Data Scientist', 'disponibilite': None}
Similarité entre Profil attendu et Profil du CV: 0.5051
